In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix

GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 15#10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
WB = 110#2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 15#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -150#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 1.0  # maximum accel [m/ss]

show_animation = True

In [2]:
def iterative_linear_mpc_control(xref, Motion_Status, dref, oa, od,x0):
    """
    MPC contorl with updating operational point iteraitvely
    """  
    
    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T
    state=Motion_Status.get_state()   
    for i in range(MAX_ITER):
        xbar = Motion_Status.get_trajectory(oa, od,T)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, state, dref,x0)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")
    
    return oa, od, ox, oy, oyaw, ov,xbar

def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)#
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)#
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
    #C = np.zeros(NX)
    return A, B, C

def linear_mpc_control(xref, xbar, state, dref,x0):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], 0)
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t]+ C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == [state.x,state.y,state.v,state.yaw]]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.OSQP, warm_start=True)
    #prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = np.array(x.value[0, :]).flatten()
        oy = np.array(x.value[1, :]).flatten()
        ov = np.array(x.value[2, :]).flatten()
        oyaw = np.array(x.value[3, :]).flatten()
        oa = np.array(u.value[0, :]).flatten()
        odelta = np.array(u.value[1, :]).flatten()

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov



In [3]:
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = curr_state.x + curr_state.v * math.cos(curr_state.yaw) * DT
        next_y = curr_state.y + curr_state.v * math.sin(curr_state.yaw) * DT
        next_yaw = curr_state.yaw + curr_state.v / WB * math.tan(delta) * DT
        next_v = curr_state.v + a * DT
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion.get_next_state(self,accel, delta, curr_state=_state)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw
            _state=next_state
        

        return xbar


In [4]:
def gen_points(cx,cy):
    dx=20#2
    dy=10#1
    pts=[[cx-dx,cy+dy],
    [cx+dx,cy+dy],
    [cx+dx,cy-dy],
    [cx-dx,cy-dy],
    [cx-dx,cy+dy]]
    pts=np.array(pts,dtype=np.float64)
    return pts
class move_points():
    def __init__(self,points,XX,YY,phi):
        self.points=points.copy()
        self.XX=XX
        self.YY=YY
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def move(self):
        self.points[:,0]+=self.XX
        self.points[:,1]+=self.YY
    def get_origin(self):
        ox=0
        oy=0
        oo=np.mean(self.points,axis=0)
        self.ox=oo[0]
        self.oy=oo[1]
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        self.points[:,0]-=np.float64(self.ox)
        self.points[:,1]-=np.float64(self.oy)
        return self.points
    def reverse_shift_origin(self):
        self.points[:,0]+=self.ox
        self.points[:,1]+=self.oy
        return self.points
    def rotate(self):
        for i in range(len(self.points)):
            self.points[i]=self.points[i].dot(self.rot_mat)
        return self.points
    def apply(self):
        self.move()
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.points

In [5]:
# initial yaw compensation
'''
if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0
'''
#goal = [cx[-1], cy[-1]]


from collections import deque
class Data_Processor():
    
    def __init__(self,_speed=1.0):
        self._stream=[]
        self.target_speed=_speed
        self.tlen=15
        self.no_of_st=4
        self.traj=np.zeros((self.no_of_st,self.tlen),dtype=np.float64)
    
    def update_trajectory(self,_tpoints):
        tx,ty=_tpoints[:,0],_tpoints[:,1]
        self.xs=np.array(tx)
        self.ys=np.array(ty)
        self.yaws=np.array(self.get_yaw())
        self.vs=np.array(self.get_speed_profile())
        
        self.traj[0,:]=self.xs
        self.traj[1,:]=self.ys
        self.traj[2,:]=self.vs
        self.traj[3,:]=self.yaws
        
        return self.traj
        
        
    def get_spline(self,tx,ty,dl):
        cx=tx
        cy=ty
        #cx, cy, _, _, _ = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
        return tx,ty
    
    def get_yaw(self):
        
        dcx=np.gradient(self.xs)
        dcy=np.gradient(self.ys)
        cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

        return cyaw
        #sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)

    def get_speed_profile(self):

        speed_profile = np.ones(len(self.xs),dtype=np.float32)
        speed_profile *=self.target_speed
        speed_profile[-1] = 0.0
        
        return speed_profile

      
    def get_ref_trajectory(self,state,_dl,_dt,t_step):
        
        traj_points=self.traj.copy()
        xref = np.zeros((NX, t_step + 1))
        dref = np.zeros((1, t_step + 1))

        xref[:, 0] = traj_points[:,0]
        dref[0, 0] = 0.0  # steer operational point should be 0

        travel = 0.0
        print(traj_points.shape[1])
        for i in range(t_step + 1):
            travel += abs(state.v) * _dt
            dind = int(round(travel / _dl))
            print('bug...........',state.v,_dt,_dl,dind,traj_points.shape[1])
            if (dind) < traj_points.shape[1]-1:
                xref[:, i] = traj_points[:,dind]
                dref[0, i] = 0.0
            else:
                print('enter',dind,len(traj_points))
                xref[:, i] = traj_points[:,-1]
                dref[0, i] = 0.0

        return xref,dref


In [6]:
# initial yaw compensation
'''
if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0
'''
#goal = [cx[-1], cy[-1]]


from collections import deque
class Map_Tracker():
    
    def __init__(self,_stream,qlen=15):
        self._stream=_stream
        self.point_stream=deque(maxlen=qlen)
        self.qlen=qlen
        self.max_len=50
        self.index=0
        self.dind=0
        self.end=False
        self.x,self.y=self._stream[:self.max_len,0],self._stream[:self.max_len,1]
        for _ in range(self.qlen):
            ind=self.index
            self.point_stream.append([self.x[ind],self.y[ind]])
            self.index+=1
            
    def add_points(self,num):
        
        for _ in range(num):
            ind=self.index
            if (ind-self.qlen-1)<len(self.x):
                self.point_stream.append([self.x[ind],self.y[ind]])
                self.index+=1
            else:
                self.end=True
                break
        return self.end
   
    def get_origin_shift(self,state):
        
        traj_points=self.point_stream.copy()
        d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy) in traj_points]
        mind = min(d)
        self.dind = d.index(mind)
        return self.dind
    
    def shift_trajectory(self):
        
        if self.dind:
            self.end=self.add_points(self.dind)
        return self.end 
    
    def run(self,state):
        self.dind=self.get_origin_shift(state)
        self.end=self.shift_trajectory()
        return np.array(self.point_stream.copy()),self.dind,self.end

In [7]:
%matplotlib tk
import gym
from matplotlib import pyplot as plt
import cv2
import numpy as np
from car_env import CarRacing
import time
import cubic_spline_planner
import pickle

class Car_Env():
    
    def __init__(self):
        self.ENV=CarRacing()
        
        self._save_path=False
        self._read_path=True
        self._state=State(x=0,y=0,yaw=0,v=0)
        self.points=[[0,0]]
        self.frame=np.zeros((100,200,3),dtype=np.uint8)
        #self.Dstr=Data_Streamer(15,TARGET_SPEED)
        
    def read_path(self):
        f=open('path-1','rb')
        track=pickle.load(f)
        f.close()
        self.ENV.track=track
    
    
    def save_path(self):
        f=open('path-2','wb')
        pickle.dump(self.ENV.track,f)
        f.close()
    
    def accl_to_gas(self,accl,vel):
        dt=1.0
        SIZE = 0.02
        ENGINE_POWER= 100000000*SIZE*SIZE
        WHEEL_MOMENT_OF_INERTIA = 4000*SIZE*SIZE
        WHEEL_R  = 27 
        wheel_rad=WHEEL_R*SIZE
        mass=(2*WHEEL_MOMENT_OF_INERTIA)/wheel_rad
        #mass=.25
        power=4*mass*accl*(2*vel+accl*dt)/2#
        #power=4*mass*accl*(vel+accl/2)
        gas=power/ENGINE_POWER
        return gas
    
    def step(self,action,show_plot=False):

       
        
        action[1]=self.accl_to_gas(action[1],self._state.v)
        #action[0]=action[0]-np.deg2rad(90)
        action[0]*=-1
        
        
        self.gas=action[1]
        self.steer=action[0]
        self.brake=action[2]
        
        nst,rw,ter,info=self.ENV.step(action) # take a random action
        self.frame[0:96,0:96,:]=np.array(nst)

        pos=self.ENV.car.hull.position
        vel=np.sqrt(self.ENV.car.hull.linearVelocity[0]**2+self.ENV.car.hull.linearVelocity[1]**2)
        ang=self.ENV.car.hull.angle
        #ang*=-1
        ang=ang+np.deg2rad(90)#-ang
        
            
        #self._state= State(x=pos[0],y=pos[1],yaw=ang,v=vel) 
        #self.Dstr.get_origin_shift(self._state)
        #self.Dstr.shift_trajectory()
        
        return [pos[0],pos[1],ang,vel],rw,ter,info

    
    
    def render(self):
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            plt.cla()
            img = cv2.putText(np.array(self.frame), 'vel %f'%vel, (100,10), font,.3, (255,255,0),1, cv2.LINE_AA)
            img = cv2.putText(np.array(img), 'ang %f'%ang, (100,20), font,.3, (255,0,255),1, cv2.LINE_AA)
            image = cv2.putText(np.array(img), 'x,y %f,%f'%(pos[0],pos[1]), (100,30), font,.2, (255,0,255),1, cv2.LINE_AA)

            plt.imshow(image)
            plt.show()
            plt.pause(0.0001)


    def reset(self):
        
        self.ENV.reset()
        
        if self._save_path==True:
            self.save_path()
        if self._read_path ==True:
            self.read_path()
        
        points=[[x,y] for a,b,x,y in self.ENV.track]
        self.points=np.array(points)
        #self.Dstr.get_planned_traj(self.points)
        
        plt.plot(self.points[:,0],self.points[:,1])
        plt.show()
        
    
       
        return [self.step([0,0,0]),self.points]


In [8]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
Env=Car_Env()
[init_state,map_points]=Env.reset()
ix,iy,iyaw,ivel=init_state[0]


Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
print(state)

Track generation: 1121..1405 -> 284-tiles track


In [9]:
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]





#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
#cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''

_Map_Tracker=Map_Tracker(map_points,qlen=15)
_Data_Processor=Data_Processor(_speed=TARGET_SPEED)

while MAX_TIME >= time:
        state=Motion_Status.get_state()
        x0 = [state.x, state.y, state.v, state.yaw]
    
        _traj,_dind,_end=_Map_Tracker.run(state)
        print("................... traj ",_traj)
        ret=_Data_Processor.update_trajectory(_traj)
        xref, dref = _Data_Processor.get_ref_trajectory(state,dl,DT,T)
        print("................... xref ",xref)
        if _end==True:
            break

        _index+=_dind
        
        oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta,x0)
        
        state_str=str(state.x)+','+str(state.y)+','+str(state.v)+','+str(np.rad2deg(state.yaw))+'\n'
        xref_str=str(xref[0][0])+','+str(xref[1][0])+','+str(xref[2][0])+','+str(np.rad2deg(xref[3][0]))+','+str(_index)+'\n'
        xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str=str(accl)+','+str(gas)+','+str(np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        if odelta is not None:
            di, ai = odelta[0], oa[0]
        accl=ai
        steer=di
        
        #act_str=str(steer)+'\n'
        nst,rw,ter,info=Env.step([steer,accl,0])
        gas=Env.gas
        
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        Motion_Status.set_state(next_state)
        
        time = time + DT

        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        
        cx=map_points[:50,0]
        cy=map_points[:50,1]
        if show_animation:  # pragma: no cover
            plt.cla()
            plt.text(50,50,'state : '+state_str,fontsize=12)
            plt.text(50,40,'xref : '+xref_str,fontsize=12)
            plt.text(50,30,'xbar : '+xbar_str,fontsize=12)
            plt.text(50,20,'act : '+act_str,fontsize=12)
            
            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")
            plt.plot(cx, cy, "-r", label="course")
            plt.plot(x, y, "ob", label="trajectory")
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")
            plt.plot(cx[_index], cy[_index], "xg", label="target")
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            points=gen_points(state.x,state.y)
            ang_yaw=-(state.yaw*180.0)/math.pi
            rp=move_points(points,0,0,ang_yaw)
            r_points=rp.apply()
            points=np.array(points,dtype=np.float64)
            r_points=np.array(r_points,dtype=np.float64)
            plt.plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            plt.title("Time[s]:" + str(round(time, 2))
                      + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
                      + '\nyaw'+str(state.yaw)
                      + '\nxref_yaw'+str(xref[3][0])
                      + '\nsteer'+str(steer))
            plt.pause(0.0001)

if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(cx, cy, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


................... traj  [[219.43452697 -55.01666274]
 [219.78940152 -51.53470007]
 [220.14427608 -48.0527374 ]
 [220.49915063 -44.57077472]
 [220.85402519 -41.08881205]
 [221.20889974 -37.60684938]
 [221.5637743  -34.12488671]
 [221.91864885 -30.64292403]
 [222.27352341 -27.16096136]
 [222.62839797 -23.67899869]
 [222.98327252 -20.19703602]
 [223.33814708 -16.71507335]
 [223.69302163 -13.23311067]
 [224.04789619  -9.751148  ]
 [224.40277074  -6.26918533]]
15
bug........... 8.836842602702243e-07 0.2 1.0 0 15
bug........... 8.836842602702243e-07 0.2 1.0 0 15
bug........... 8.836842602702243e-07 0.2 1.0 0 15
bug........... 8.836842602702243e-07 0.2 1.0 0 15
bug........... 8.836842602702243e-07 0.2 1.0 0 15
bug........... 8.836842602702243e-07 0.2 1.0 0 15
................... xref  [[219.43452697 219.43452697 219.43452697 219.43452697 219.43452697
  219.43452697]
 [-55.01666274 -55.01666274 -55.01666274 -55.01666274 -55.01666274
  -55.01666274]
 [ 15.          15.          15.          1

214.81906127929688 -52.6611213684082
................... traj  [[219.43452697 -55.01666274]
 [219.78940152 -51.53470007]
 [220.14427608 -48.0527374 ]
 [220.49915063 -44.57077472]
 [220.85402519 -41.08881205]
 [221.20889974 -37.60684938]
 [221.5637743  -34.12488671]
 [221.91864885 -30.64292403]
 [222.27352341 -27.16096136]
 [222.62839797 -23.67899869]
 [222.98327252 -20.19703602]
 [223.33814708 -16.71507335]
 [223.69302163 -13.23311067]
 [224.04789619  -9.751148  ]
 [224.40277074  -6.26918533]]
15
bug........... 0.04865816535856623 0.2 1.0 0 15
bug........... 0.04865816535856623 0.2 1.0 0 15
bug........... 0.04865816535856623 0.2 1.0 0 15
bug........... 0.04865816535856623 0.2 1.0 0 15
bug........... 0.04865816535856623 0.2 1.0 0 15
bug........... 0.04865816535856623 0.2 1.0 0 15
................... xref  [[219.43452697 219.43452697 219.43452697 219.43452697 219.43452697
  219.43452697]
 [-55.01666274 -55.01666274 -55.01666274 -55.01666274 -55.01666274
  -55.01666274]
 [ 15.          15

214.8203125 -52.65011978149414
................... traj  [[219.43452697 -55.01666274]
 [219.78940152 -51.53470007]
 [220.14427608 -48.0527374 ]
 [220.49915063 -44.57077472]
 [220.85402519 -41.08881205]
 [221.20889974 -37.60684938]
 [221.5637743  -34.12488671]
 [221.91864885 -30.64292403]
 [222.27352341 -27.16096136]
 [222.62839797 -23.67899869]
 [222.98327252 -20.19703602]
 [223.33814708 -16.71507335]
 [223.69302163 -13.23311067]
 [224.04789619  -9.751148  ]
 [224.40277074  -6.26918533]]
15
bug........... 0.09540063821318782 0.2 1.0 0 15
bug........... 0.09540063821318782 0.2 1.0 0 15
bug........... 0.09540063821318782 0.2 1.0 0 15
bug........... 0.09540063821318782 0.2 1.0 0 15
bug........... 0.09540063821318782 0.2 1.0 0 15
bug........... 0.09540063821318782 0.2 1.0 0 15
................... xref  [[219.43452697 219.43452697 219.43452697 219.43452697 219.43452697
  219.43452697]
 [-55.01666274 -55.01666274 -55.01666274 -55.01666274 -55.01666274
  -55.01666274]
 [ 15.          15.     

214.82241821289062 -52.63174819946289
................... traj  [[219.43452697 -55.01666274]
 [219.78940152 -51.53470007]
 [220.14427608 -48.0527374 ]
 [220.49915063 -44.57077472]
 [220.85402519 -41.08881205]
 [221.20889974 -37.60684938]
 [221.5637743  -34.12488671]
 [221.91864885 -30.64292403]
 [222.27352341 -27.16096136]
 [222.62839797 -23.67899869]
 [222.98327252 -20.19703602]
 [223.33814708 -16.71507335]
 [223.69302163 -13.23311067]
 [224.04789619  -9.751148  ]
 [224.40277074  -6.26918533]]
15
bug........... 0.14137152946761478 0.2 1.0 0 15
bug........... 0.14137152946761478 0.2 1.0 0 15
bug........... 0.14137152946761478 0.2 1.0 0 15
bug........... 0.14137152946761478 0.2 1.0 0 15
bug........... 0.14137152946761478 0.2 1.0 0 15
bug........... 0.14137152946761478 0.2 1.0 0 15
................... xref  [[219.43452697 219.43452697 219.43452697 219.43452697 219.43452697
  219.43452697]
 [-55.01666274 -55.01666274 -55.01666274 -55.01666274 -55.01666274
  -55.01666274]
 [ 15.          1

214.82542419433594 -52.60613250732422
................... traj  [[219.43452697 -55.01666274]
 [219.78940152 -51.53470007]
 [220.14427608 -48.0527374 ]
 [220.49915063 -44.57077472]
 [220.85402519 -41.08881205]
 [221.20889974 -37.60684938]
 [221.5637743  -34.12488671]
 [221.91864885 -30.64292403]
 [222.27352341 -27.16096136]
 [222.62839797 -23.67899869]
 [222.98327252 -20.19703602]
 [223.33814708 -16.71507335]
 [223.69302163 -13.23311067]
 [224.04789619  -9.751148  ]
 [224.40277074  -6.26918533]]
15
bug........... 0.18661130291595251 0.2 1.0 0 15
bug........... 0.18661130291595251 0.2 1.0 0 15
bug........... 0.18661130291595251 0.2 1.0 0 15
bug........... 0.18661130291595251 0.2 1.0 0 15
bug........... 0.18661130291595251 0.2 1.0 0 15
bug........... 0.18661130291595251 0.2 1.0 0 15
................... xref  [[219.43452697 219.43452697 219.43452697 219.43452697 219.43452697
  219.43452697]
 [-55.01666274 -55.01666274 -55.01666274 -55.01666274 -55.01666274
  -55.01666274]
 [ 15.          1

...

Iterative is max iter
171.01470947265625 39.627471923828125
................... traj  [[174.39826176  38.23271229]
 [171.31658064  39.89200175]
 [168.23489953  41.55129122]
 [165.15321842  43.21058068]
 [162.07153731  44.86987015]
 [158.9898562   46.52915961]
 [155.90817509  48.18844908]
 [152.82649398  49.84773854]
 [149.74481287  51.50702801]
 [146.66313176  53.16631747]
 [143.58145064  54.82560694]
 [140.49976953  56.4848964 ]
 [137.41808842  58.14418587]
 [134.33640731  59.80347534]
 [131.2547262   61.4627648 ]]
15
bug........... 4.599562959509485 0.2 1.0 1 15
bug........... 4.599562959509485 0.2 1.0 2 15
bug........... 4.599562959509485 0.2 1.0 3 15
bug........... 4.599562959509485 0.2 1.0 4 15
bug........... 4.599562959509485 0.2 1.0 5 15
bug........... 4.599562959509485 0.2 1.0 6 15
................... xref  [[171.31658064 168.23489953 165.15321842 162.07153731 158.9898562
  155.90817509]
 [ 39.89200175  41.55129122  43.21058068  44.86987015  46.52915961
   48.18844908]
 [ 15.  

170.37013244628906 39.98509216308594
................... traj  [[174.39826176  38.23271229]
 [171.31658064  39.89200175]
 [168.23489953  41.55129122]
 [165.15321842  43.21058068]
 [162.07153731  44.86987015]
 [158.9898562   46.52915961]
 [155.90817509  48.18844908]
 [152.82649398  49.84773854]
 [149.74481287  51.50702801]
 [146.66313176  53.16631747]
 [143.58145064  54.82560694]
 [140.49976953  56.4848964 ]
 [137.41808842  58.14418587]
 [134.33640731  59.80347534]
 [131.2547262   61.4627648 ]]
15
bug........... 4.616835851916595 0.2 1.0 1 15
bug........... 4.616835851916595 0.2 1.0 2 15
bug........... 4.616835851916595 0.2 1.0 3 15
bug........... 4.616835851916595 0.2 1.0 4 15
bug........... 4.616835851916595 0.2 1.0 5 15
bug........... 4.616835851916595 0.2 1.0 6 15
................... xref  [[171.31658064 168.23489953 165.15321842 162.07153731 158.9898562
  155.90817509]
 [ 39.89200175  41.55129122  43.21058068  44.86987015  46.52915961
   48.18844908]
 [ 15.          15.          15

169.7230224609375 40.34383773803711
................... traj  [[174.39826176  38.23271229]
 [171.31658064  39.89200175]
 [168.23489953  41.55129122]
 [165.15321842  43.21058068]
 [162.07153731  44.86987015]
 [158.9898562   46.52915961]
 [155.90817509  48.18844908]
 [152.82649398  49.84773854]
 [149.74481287  51.50702801]
 [146.66313176  53.16631747]
 [143.58145064  54.82560694]
 [140.49976953  56.4848964 ]
 [137.41808842  58.14418587]
 [134.33640731  59.80347534]
 [131.2547262   61.4627648 ]]
15
bug........... 4.63448435385222 0.2 1.0 1 15
bug........... 4.63448435385222 0.2 1.0 2 15
bug........... 4.63448435385222 0.2 1.0 3 15
bug........... 4.63448435385222 0.2 1.0 4 15
bug........... 4.63448435385222 0.2 1.0 5 15
bug........... 4.63448435385222 0.2 1.0 6 15
................... xref  [[171.31658064 168.23489953 165.15321842 162.07153731 158.9898562
  155.90817509]
 [ 39.89200175  41.55129122  43.21058068  44.86987015  46.52915961
   48.18844908]
 [ 15.          15.          15.      

169.0734100341797 40.7039794921875
................... traj  [[174.39826176  38.23271229]
 [171.31658064  39.89200175]
 [168.23489953  41.55129122]
 [165.15321842  43.21058068]
 [162.07153731  44.86987015]
 [158.9898562   46.52915961]
 [155.90817509  48.18844908]
 [152.82649398  49.84773854]
 [149.74481287  51.50702801]
 [146.66313176  53.16631747]
 [143.58145064  54.82560694]
 [140.49976953  56.4848964 ]
 [137.41808842  58.14418587]
 [134.33640731  59.80347534]
 [131.2547262   61.4627648 ]]
15
bug........... 4.652206445362878 0.2 1.0 1 15
bug........... 4.652206445362878 0.2 1.0 2 15
bug........... 4.652206445362878 0.2 1.0 3 15
bug........... 4.652206445362878 0.2 1.0 4 15
bug........... 4.652206445362878 0.2 1.0 5 15
bug........... 4.652206445362878 0.2 1.0 6 15
................... xref  [[171.31658064 168.23489953 165.15321842 162.07153731 158.9898562
  155.90817509]
 [ 39.89200175  41.55129122  43.21058068  44.86987015  46.52915961
   48.18844908]
 [ 15.          15.          15. 

IndexError: index 50 is out of bounds for axis 0 with size 50

In [ ]:
ret

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
i_yaw=cyaw[0]
cyaw=[yaw-i_yaw for yaw in cyaw]
cyaw

In [ ]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [ ]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [ ]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
yaw0=np.deg2rad(90)
cyaw=[yaw-yaw0 for yaw in cyaw]

In [ ]:
import numpy as np
np.rad2deg(2.64)

In [ ]:
cyaw

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 1
n = 1
np.random.seed(1)
A = 10#np.random.randn(m, n)
b = 5#np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A*x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

In [ ]:
print(x.value)

In [ ]:
x=np.array(range(0,100))
x=x/100

In [ ]:
y=10*x-5
y=np.square(y)
plt.plot(x,y)
plt.show()